In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

# Список республик бывшего СССР
resp = ['Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Lithuania',\
        'Republic of Moldova', 'Russian Federation', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan']

Для начала посмотрим статистику по "Ожидаемая продолжительность жизни". Возьмем информацию только для стран бывшего СССР, только за 2019 и общую для обеих полов.

In [116]:
url = "./dataframes/lifeExpectancyAtBirth.csv"
data_leab = pd.read_csv(url, usecols=['country','period','gender','age'], \
                        dtype = {'country': str, 'period': int, 'gender': str, 'age': float},\
                        header=None, skiprows=[0,], names=['country','period','indicator','gender','age'])
data_leab = data_leab[(data_leab['country'].isin(resp)) & (data_leab['gender']=='Both sexes') &\
                      (data_leab['period']==2019)]
data_leab[['country','age']].sort_values(by='age', ascending=False)

,country,age
649,Estonia,78.88
72,Armenia,76.03
1153,Lithuania,75.99
1093,Latvia,75.38
168,Belarus,74.81
1069,Kyrgyzstan,74.18
1021,Kazakhstan,73.95
745,Georgia,73.28
1585,Republic of Moldova,73.26
1621,Russian Federation,73.23


Как можно увидеть по таблице выше на первых местах по продолжительности жизни расположились Прибалтийские страны (Эстония, Латвия, Литва) и Армения. На последних местах Азербайджан, Туркменистан и Таджикистан. Россия в списке находится на 10 месте.
<br><br>
Попробуем понять, что на это влияет. Для начала рассмотрим уровень медицины в этих странах и насколько он различается.
1. Сравним какое кол-во докторов приходится на каждые  на 10 000 населения.
<br>
(т.к. данные по всем странам бывшего СССР есть только за 2014 год, то для сравнения возьмем данные именно за этот год)

In [135]:
url = "./dataframes/medicalDoctors.csv"
data_mcd = pd.read_csv(url, usecols=['country','period','doctors'], \
                        dtype = {'country': str, 'Period': int, 'gender': str, 'doctors': float},\
                        header=None, skiprows=[0,], names=['country','period','indicator','doctors'])
data_mcd = data_mcd[(data_mcd['country'].isin(resp)) & (data_mcd['period']==2014)]
data_mcd[['country','doctors']].sort_values(by='doctors', ascending=False)

,country,doctors
206,Belarus,50.12
844,Georgia,47.75
1316,Lithuania,42.51
1871,Russian Federation,40.11
1176,Kazakhstan,39.80
138,Azerbaijan,34.46
726,Estonia,33.56
1262,Latvia,31.72
2344,Ukraine,29.92
70,Armenia,28.93


Как можно увидеть из таблицы выше, кол-во врачей прямо не коррелируются с продолжительностью жизни. Единственное совпадение это Туркменистан и Таджикистан с наименьшим кол-вом врачей на 10 т.ч. и с наименьшой продолжительностю жизни среди других стран.
<br>
2. Теперь посмотрим как обстоит дело с медсестринским и акушерским персоналом на 10 000 населения.

In [140]:
url = "./dataframes/nursingAndMidwife.csv"
data_nam = pd.read_csv(url, usecols=['country','period','doctors'], \
                        dtype = {'country': str, 'Period': int, 'gender': str, 'doctors': float},\
                        header=None, skiprows=[0,], names=['country','period','indicator','doctors'])
data_nam = data_nam[(data_nam['country'].isin(resp)) & (data_nam['period']==2014)]
data_nam[['country','doctors']].sort_values(by='doctors', ascending=False)

,country,doctors
2510,Uzbekistan,112.80
200,Belarus,109.80
1361,Lithuania,78.05
1205,Kazakhstan,72.00
2435,Ukraine,66.61
134,Azerbaijan,64.34
772,Estonia,60.72
1263,Kyrgyzstan,59.45
1884,Republic of Moldova,53.11
75,Armenia,51.55


Здесь также нельзя провести прямую связь между продолжительностью жизни и кол-вом мед.персонала.
<br>
Попробуем предположить, что на это влияют различные факторы в стране, такие как чистая питьевая вода, санитария и гигиена.
<br>
3. Посмотрим какой процент населения пользующется хотя бы основными услугами питьевой воды

In [142]:
url = "./dataframes/basicDrinkingWaterServices.csv"
data_nam = pd.read_csv(url, usecols=['country','period','percent'], \
                        dtype = {'country': str, 'Period': int, 'gender': str, 'percent': float},\
                        header=None, skiprows=[0,], names=['country','period','indicator','percent'])
data_nam = data_nam[(data_nam['country'].isin(resp)) & (data_nam['period']==2016)]
data_nam[['country','percent']].sort_values(by='percent', ascending=False)

,country,percent
127,Armenia,100.00
1023,Estonia,99.19
1689,Latvia,98.03
3169,Turkmenistan,97.72
271,Belarus,95.96
3330,Uzbekistan,95.86
1167,Georgia,95.71
2556,Russian Federation,92.14
1779,Lithuania,91.74
3222,Ukraine,91.32


Здесь уже можно провести некую связь между людьми с доступом к питьевой воде и продолжительностю жизни.
<br>
На первых 3 местах мы можем видеть Армению, Эстонию и Латвию. Азербайджан и Таджикистан на 2-х последних.